Here we are working with large datasets in PySpark, and our job keeps failing due to memory errors(OutOfMemoryError) during wide transformation like groupBy and joins. create a solution to optimize the PySpark job to reduce memory pressure and prevent job failure,while still performing a groupBy aggregation on a large datasets.
Sales_data with following columns like store_id,product_id,sales,amount,sale_date. we have to calculate the total sales per store per day.But the data is huge and we want to ensure it runs efficiently in production without memory isssues...

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
from datetime import date

Here we are creating schema and table with various columns

In [29]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, DateType
data = [
    (1, 101, 150.75, date(2024, 11, 1)),
    (1, 102, 200.00, date(2024, 11, 2)),
    (2, 103, 350.50, date(2024, 11, 2)),
    (2, 101, 125.00, date(2024, 11, 3)),
    (3, 104, 500.00, date(2024, 11, 4))
]

schema = StructType([
    StructField("store_id", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("sales_amount", DoubleType(), True),
    StructField("sale_date", DateType(), True)
])


In [30]:
spark=SparkSesion.builder.appName("memoryError").config("spark.sql.shuffle.partitions","200").config("spark.sql.autoBroadcastJoinThreshold",-1).getOrCreate()


In [31]:
columns=["store_id", "product_id", "sales_amount", "sale_date"]

In [38]:
sales_df=spark.createDataFrame(data,columns)

In [39]:
sales_df.select('store_id')

DataFrame[store_id: bigint]

Load data(assume from parquet or partitioned source

In [40]:
# sales_df=spark.read.csv("store_id")
# sales_df


Repartition before wide transformation to optimize memory usage


In [41]:
sales_df=sales_df.repartition("store_id")

Perform Aggregation

In [42]:
result_df=sales_df.groupBy("store_id","sale_date").agg(sum("sales_amount").alias("total_sales"))

In [43]:
result_df.show()

+--------+----------+-----------+
|store_id| sale_date|total_sales|
+--------+----------+-----------+
|       1|2024-11-01|     150.75|
|       1|2024-11-02|      200.0|
|       3|2024-11-04|      500.0|
|       2|2024-11-02|      350.5|
|       2|2024-11-03|      125.0|
+--------+----------+-----------+

